
# Sockets Calculator tutorial


An introduction to the Sockets IO interface in ASE; more expansive API documentation is available here:
https://wiki.fysik.dtu.dk/ase/ase/calculators/socketio/socketio.html

The starting point is understanding the standard structure of an ASE calculation when you use a "normal" calculator defined for example as:

In [1]:
from ase.calculators.aims import Aims

fhi_calc = Aims(xc='pbe')

This calc is then attached to any atoms object for energy/force/stress calculations

In [2]:
from ase.io import read

atoms = read('../../data/Fe-CO/fe-co_light.log')

atoms.set_calculator(fhi_calc)

Whenever you want to perform an energy/force/stress calculation with this `calculator` attached to an `atoms` object, the following process occurs:

Write input file to disk -> Call MPI executable -> Run program -> Read output file from disk
<!-- This would be much better as an image //-->

The reading and writing of data, known as input/output or I/O, is time consuming; it's much more efficient if data can be transfered directly between executables:

Call MPI executable -> Give input data -> Run program -> Return output data.
<!-- This would be much better as an image //-->

Sockets are a method of interfacing executables exactly for the purpose of avoiding data I/O and repeated `system` calls to run the MPI executable. 

In the simplest form we can create a `SocketsIOCalculator` thus:

In [3]:
from ase.calculators.socketio import SocketIOCalculator

port = 12345

sockets_calc = SocketIOCalculator(calc=fhi_calc,  
                                  port=port)

FHI-aims *needs to know* that we want to use the sockets connection, so we need to give it information on how we will connect also. We can edit any calculator parameter with `set()`:

In [4]:
fhi_calc.set(use_pimd_wrapper=('localhost', port))

{'use_pimd_wrapper': ('localhost', 12345)}

Now we can attach the `sockets_calc` to our atoms objects for any calculation:

In [5]:
atoms.set_calculator(sockets_calc)

When using FHI-aims in this manner, the MPI execution remains the same (`mpirun -np ....`) but it is important to know that the executable will **not** close at the end of the first calculation; instead, *it will remain active*, waiting for further commands. Therefore it's important to close everything down at the end of a calculation with:

In [6]:
sockets_calc.close()

An alternative is to use the `with` statement for temporary use of a variable:

In [7]:
with SocketIOCalculator(calc=fhi_calc,  port=port) as calc:
    
    atoms.set_calculator(sockets_calc)
    
    #... create opt/dyn object and run calculations ...
    
#... and then continue

With everything combined together, a full example script might look like:

In [8]:
from ase import Atoms
from ase.calculators.aims import Aims
from ase.calculators.socketio import SocketIOCalculator
from ase.optimize import BFGS 

# Set Sockets port
port = 12345

# Build molecule
water = Atoms('HOH', [(1, 0, 0), (0, 0, 0), (0, 1, 0)])

# Basic method to setup FHI-aims up calculator
fhi_calc = Aims(xc='PBE',
                use_pimd_wrapper=('localhost', port))

# Use the sockets calculator
with SocketIOCalculator(fhi_calc, port=port) as sockets_calc:

    water.set_calculator(sockets_calc)

    # Setup optimisation
    dynamics = BFGS(water, trajectory='square_water.traj')

    # Run optimisation. Enable only if you actually have the FHI-aims binary set correctly!
    #dynamics.run(fmax=0.01)

There are a couple of extra technical notes to make for different systems:

- **On computers where Python runs on the work nodes, alongside the MPI** (e.g. Hawk), the sockets calculator can be setup as in these examples with the connecting host being ``localhost``

- **On computers where Python runs on launcher nodes, different to the MPI** (e.g. Isambard, ARCHER), the FHI calculator *needs* to know the name of the launcher node where it will receive a connection from. This can be simplly picked up by:

In [9]:
import socket

hostname = socket.gethostname()

fhi_calc.set(use_pimd_wrapper=(hostname, port))

{'use_pimd_wrapper': ('Andrews-MacBook-Pro.local', 12345)}

Finally, all of the settings can be simplified by using some of the functionality embedded into `carmm`. Instead of building and connecting the `Aims` and `SocketsIO` calculators, there is a function available that returns these default, connected calculators for the user (factoring in hostnames if necessary):

In [1]:
from carmm.run.aims_calculator import get_aims_and_sockets_calculator

# Example when ASE and MPI run on the same nodes. Dimensions=0 means a molecular system
sockets_calc, fhi_calc = get_aims_and_sockets_calculator(dimensions=0)

#... do some work ...

# At the end of a sockets calculation, you MUST close the calculator for safety
sockets_calc.close()

Remember **always** in this case to personalise the FHI-aims settings.